In [88]:
import numpy as np

np.set_printoptions(threshold=10000, suppress=True)
import pandas as pd
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [89]:
data = pd.read_csv('PubMed-multi-label-dataset.csv', sep=',', encoding="utf-8")
data.head(3)

,Title,abstractText,meshMajor,pmid,meshid,meshroot,A,B,C,D,E,F,G,H,I,J,L,M,N,Z
0,Expression of p53 and coexistence of HPV in pr...,Fifty-four paraffin embedded tissue sections f...,"['DNA Probes, HPV', 'DNA, Viral', 'Female', 'H...",8549602,"[['D13.444.600.223.555', 'D27.505.259.750.600....","['Chemicals and Drugs [D]', 'Organisms [B]', '...",0,1,1,1,1,0,0,1,0,0,0,0,0,0
1,Vitamin D status in pregnant Indian women acro...,The present cross-sectional study was conducte...,"['Adult', 'Alkaline Phosphatase', 'Breast Feed...",21736816,"[['M01.060.116'], ['D08.811.277.352.650.035'],...","['Named Groups [M]', 'Chemicals and Drugs [D]'...",0,1,1,1,1,1,1,0,1,1,0,1,1,1
2,[Identification of a functionally important di...,The occurrence of individual amino acids and d...,"['Amino Acid Sequence', 'Analgesics, Opioid', ...",19060934,"[['G02.111.570.060', 'L01.453.245.667.060'], [...","['Phenomena and Processes [G]', 'Information S...",1,1,0,1,1,0,1,0,0,0,1,0,0,0


In [90]:
corpus = data['abstractText']

### Proposer un code qui permettra d’apprendre votre propre modèle de plongement lexical Word2Vec sur vos données textuelles. Évaluer visuellement et numériquement sur quelques mots clés votre nouveau modèle de vectorisation (Embedding).

In [ ]:
import numpy as np
from collections import defaultdict
from nltk.tokenize import word_tokenize

# Tokeniser les phrases
tokenized_corpus = [word_tokenize(sentence.lower()) for sentence in corpus]

# Définir les paramètres du modèle Word2Vec
vector_size = 100
window_size = 2
learning_rate = 0.025
epochs = 10

# Initialiser les vecteurs de mots
word_vectors = defaultdict(lambda: np.random.uniform(-0.25, 0.25, vector_size))

# Fonction pour mettre à jour les vecteurs de mots
def update_word_vector(target, context, learning_rate):
    error = word_vectors[context] - word_vectors[target]
    word_vectors[target] += learning_rate * error
    word_vectors[context] -= learning_rate * error

# Entraîner le modèle Word2Vec
for epoch in range(epochs):
    for sentence in tokenized_corpus:
        for target_index, target_word in enumerate(sentence):
            for context_word in sentence[max(0, target_index - window_size):target_index + window_size]:
                if target_word != context_word:
                    update_word_vector(target_word, context_word, learning_rate)

# Exemple d'utilisation des vecteurs de mots après l'entraînement
print(word_vectors["exemple"])


### Exploiter votre modèle Word2Vec pour la vectorisation de vos textes (avec deux méthodes utilisant ou non le TF-IDF des mots). Exécuter ensuite à nouveau votre fonction run_models sur vos données vectorisées par Word2Vec et interpréter les résultats obtenus en les comparant à ceux obtenus aux étapes précédentes. Vous pouvez vous inspirer de la fonction suivante :

In [85]:
## get word2vec for each sentence by using average word embeddings
def word2vec_generator(texts,model,vector_size): 
    dict_word2vec = {}
    for index, word_list in enumerate(texts):
        arr = np.array([0.0 for i in range(0, vector_size)]) 
        nb_word=0
        for word in word_list:
            try:
                arr += model[word] 
                nb_word=nb_word+1
            except KeyError: 
                continue
        if(len(word_list) == 0): 
            dict_word2vec[index] = arr
        else:
            dict_word2vec[index] = arr / nb_word
    df_word2vec = pd.DataFrame(dict_word2vec).T 
    return df_word2vec

In [ ]:
df_word2vec = word2vec_generator(tokenized_corpus,word_vectors,vector_size)

In [29]:
import gensim

## Apprentissage du Word2vec

Proposer un code qui permettra d’apprendre votre propre modèle de plongement lexical Word2Vec sur vos données textuelles. Évaluer visuellement et numériquement sur quelques mots clés votre nouveau modèle de vectorisation (Embedding).

In [30]:
corpus = corpus.apply(lambda line: gensim.utils.simple_preprocess((line)))

In [32]:
#Ici le code suivant a besoin de votre corpus de texte où chaque texte est tokenizé (une liste de tokens)

In [33]:
import multiprocessing

cores = multiprocessing.cpu_count()
cores

8

In [34]:
model_size = 100
model = gensim.models.Word2Vec(corpus, vector_size=model_size, sg=0, window=5, min_count=2, workers=cores - 1)

In [35]:
for i in range(100):
    model.train(corpus, total_examples=len(corpus), epochs=1)
    print(i, end=' ')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 

In [37]:
model.save('models/Word2vec_entraine.h5')

In [38]:
model = gensim.models.Word2Vec.load('models/Word2vec_entraine.h5')

In [44]:
len(model.wv.index_to_key)

74651

In [40]:
model.wv['easy']

array([ 1.6654079 , -1.5034784 ,  0.5110981 , -4.1930223 ,  0.09314937,
        2.7616625 ,  0.04700468, -2.2822714 ,  0.03874538, -0.52443343,
       -2.8938997 ,  2.148002  , -1.9232304 , -0.06286446,  3.1893587 ,
       -5.270746  ,  0.32784304, -1.9411395 , -2.1714635 , -1.4288378 ,
       -1.5348532 ,  1.473419  , -2.1207361 ,  0.49881884, -2.9406567 ,
       -2.5208883 ,  2.3622885 ,  0.20641853, -0.25003427,  2.7113533 ,
        0.06382579, -4.113934  , -1.3935217 ,  1.0365883 ,  2.7860017 ,
       -0.5919349 , -0.7498196 ,  0.33379614,  1.523776  , -3.8976786 ,
        2.3132668 , -0.19561335,  1.0088341 , -0.07472833,  3.0608702 ,
        0.38650712, -3.7923768 ,  0.17576897, -0.504891  ,  0.8269976 ,
       -0.7221326 , -0.40463012,  0.8607108 ,  0.24683255, -1.3537418 ,
        0.23207827,  4.110492  ,  0.13510512,  4.179361  ,  1.4840094 ,
       -1.1553626 , -1.1207767 , -0.1877254 , -0.877413  ,  0.03301156,
        0.65108037, -2.92177   , -0.1319233 ,  1.240849  ,  1.33

In [53]:
model.wv.most_similar(positive='thanks', topn=10)

[('owing', 0.5255916118621826),
 ('ensure', 0.5067655444145203),
 ('accommodate', 0.5008588433265686),
 ('preserve', 0.4975113570690155),
 ('accomplish', 0.4673018157482147),
 ('due', 0.4569520950317383),
 ('pertaining', 0.4540463984012604),
 ('ascribe', 0.4397198259830475),
 ('itinerary', 0.4372483491897583),
 ('navigate', 0.43124091625213623)]

In [51]:
model.wv.doesnt_match(['food', 'drink', 'play'])

'play'

In [84]:
model.wv.most_similar(positive=['blood', 'glucose'], topn=1)

[('lactate', 0.6244521141052246)]